<a href="https://colab.research.google.com/github/sincendz/mineracao-de-dados/blob/main/02.E0-Exercicio-Classificacao-de-Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/paulotguerra/QXD0178/blob/main/02.E0-Exercicio-Classificacao-de-dados.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## QXD0178 - Mineração de Dados
# Classificação de dados

**Professor:** Paulo de Tarso Guerra Oliveira ([paulodetarso@ufc.br](mailto:paulodetarso@ufc.br))


# Lista de Exercícios: Classificação de dados

Nesta lista de exercícios, você explorará a aplicação de métodos de aprendizado de máquina para realizar tarefas de classificação de dados. Você usará a base de dados [Food choices: College students' food and cooking preferences](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) e avaliará vários algoritmos de classificação para determinar sua eficácia. O objetivo é entender como diferentes métodos de aprendizado de máquina se comportam em relação à acurácia na classificação de dados.

O exercício será dividido em várias etapas:

1. **Pré-processamento dos dados:**
   - Descreva brevemente o conjunto de dados   
   - Limpe o conjunto de dados, tratando valores ausentes, removendo duplicatas e realizando transformações necessárias.
   - Caso você use os dados pré-processados na lista anterior, faça um breve descritivo dos principais ajustes.
   - Codifique variáveis categóricas, se necessário, para que possam ser utilizadas em algoritmos de aprendizado de máquina.
   - Cria a coluna `self_perception_overweight` com valor: `True` se a coluna `self_perception_weight` tem valor 4 ou 5; e `False`, caso contrário.
   - Remova a coluna `self_perception_weight` do conjunto de dados.
2. **Divisão do conjunto de dados:**
   - Divida o conjunto de dados em um conjunto de treinamento e um conjunto de teste para avaliar o desempenho dos algoritmos.
   - O mesmo conjunto de teste deve ser usado por todos os algoritmos analizados e nenhum dado deste pode ser usado na fase de treinamento.
   - O atributo alvo (*rótulo*) da classificação será o campo `self_perception_overweight`.   
3. **Seleção de algoritmos de classificação:**
   - Selecione uma variedade de algoritmos de aprendizado de máquina para testar na tarefa de classificação.   
   - Sua seleção deve conter, no mínimo, os seguintes métodos: Naive Bayes, k-Nearest Neighbors, Support Vector Machine (Linear/RBF), Decision Trees, Random Forest, Multilayer Perceptron.
   - Descreva brevemente como funciona cada algoritmo selecionado.
4. **Treinamento e avaliação:**
   - Treine os algoritmos de classificação usando todo o conjunto de treinamento.
   - Avalie o desempenho de cada algoritmo no conjunto de teste usando métricas como acurácia, precisão, recall e F1-score.
   - Repita a análise treinando os algoritmos com validação cruzada.
   - Repita a análise realizando ajuste de hiperparâmetros.
5. **Análise dos resultados:**
   - Prepare um texto que descreva os resultados obtidos e faça uma análise crítica destes resultados.
   - Compare o desempenho dos diferentes algoritmos e explique por que alguns apresentaram resultados mais adequados que outros.
   
Documente todas as etapas em um arquivo Jupyter Notebook (`.ipynb`) que inclua as análises, o código e as justificativas. Lembre-se de que é fundamental justificar todas as decisões tomadas ao longo do processo e documentar as análises de forma clara e concisa. Este trabalho tem como objetivo proporcionar uma compreensão prática da seleção e avaliação de algoritmos de classificação em cenários de aprendizado supervisionado.

Envie seu Jupyter Notebook até a data de entrega especificada nesta tarefa.

## Solução


### Pré-processamento dos dados

In [27]:
import re
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report






df = pd.read_csv("https://raw.githubusercontent.com/She-Codes-Now/Intro-to-Data-Science-with-R/master/food_coded.csv")
#df

In [2]:
def is_number(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

#Pega um número de dentro de uma string
def extrair_numeros(string):
    lista = re.findall(r'\d+\.\d+|\d+', string)
    if len(lista) == 0:
        return np.nan
    else:
        return float(lista[0])
    return lista

# Função para converter a coluna para valores numéricos
def to_numeric(df):
    return df.map(lambda x: float(x) if is_number(x) else extrair_numeros(x))

def to_lower_case(df):
    return df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

def replace_slash_with_comma(df):
    return df.apply(lambda x: x.str.replace('/', ',') if x.dtype == 'object' else x)
def replace_r_with_comma(df):
    return df.apply(lambda x: x.str.replace('\r', ',') if x.dtype == 'object' else x)

In [3]:
gpa = ['GPA']
weight = ['weight']


# Criando o transformer para a coluna GPA
gpa_transformer = Pipeline(steps=[
    ('to_numeric', FunctionTransformer(to_numeric)),
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
]).set_output(transform='pandas')


weight_transformer = Pipeline(steps=[
    ('to_numeric', FunctionTransformer(to_numeric)),
    ('imputer', SimpleImputer(strategy='mean')),
],).set_output(transform='pandas')


preprocessing = ColumnTransformer(transformers=[
    ('GPA', gpa_transformer, ['GPA']),
    ('weight', weight_transformer, ['weight']),
], remainder='passthrough').set_output(transform='pandas')

df = preprocessing.fit_transform(df)


In [4]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Identificar as colunas do tipo object (geralmente colunas de texto ou categóricas)
object_columns = df.select_dtypes(include=['object']).columns.tolist()


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

string_transformer = Pipeline(steps=[
    ('to_lower_case', FunctionTransformer(to_lower_case)),
    ('replace_slash_with_comma', FunctionTransformer(replace_slash_with_comma)),
    ('replace_r_with_comma', FunctionTransformer(replace_r_with_comma)),
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Usando ColumnTransformer para aplicar a transformação na coluna GPA
preprocessing = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_columns),  # Aplica a transformação nas colunas numéricas
    ('object', string_transformer, object_columns)
], remainder='passthrough').set_output(transform='pandas')


# Aplicando a transformação no DataFrame
df2 = preprocessing.fit_transform(df)



In [5]:
#df2['numeric__remainder__self_perception_weight']
df2['self_perception_overweight'] = df2['numeric__remainder__self_perception_weight'].apply(lambda x: True if x == 4 or x == 5 else False)
# Change the following line to drop the column by name
df2 = df2.drop(columns=['numeric__remainder__self_perception_weight'])
df2

,numeric__GPA__GPA,numeric__weight__weight,numeric__remainder__Gender,numeric__remainder__breakfast,numeric__remainder__calories_chicken,numeric__remainder__calories_day,numeric__remainder__calories_scone,numeric__remainder__coffee,numeric__remainder__comfort_food_reasons_coded,numeric__remainder__cook,...,object__remainder__eating_changes,object__remainder__father_profession,object__remainder__fav_cuisine,object__remainder__food_childhood,object__remainder__healthy_meal,object__remainder__ideal_diet,object__remainder__meals_dinner_friend,object__remainder__mother_profession,object__remainder__type_sports,self_perception_overweight
0,0.111111,187.00000,2.0,1.0,430.0,3.0,315.0,1.0,9.0,2.0,...,eat faster,profesor,arabic cuisine,rice and chicken,looks not oily,being healthy,"rice, chicken, soup",unemployed,car racing,False
1,0.807778,155.00000,1.0,1.0,610.0,3.0,420.0,2.0,1.0,3.0,...,i eat out more than usual.,self employed,italian,"chicken and biscuits, beef soup, baked beans","grains, veggies, (more of grains and veggies),...",try to eat 5-6 small meals a day. while trying...,"pasta, steak, chicken",nurse rn,basketball,False
2,0.611111,159.04918,1.0,1.0,720.0,4.0,420.0,2.0,1.0,1.0,...,sometimes choosing to eat fast food instead of...,owns business,italian,"mac and cheese, pizza, tacos",usually includes natural ingredients; nonproce...,i would say my ideal diet is my current diet,"chicken and rice with veggies, pasta, some kin...",owns business,none,False
3,0.555556,240.00000,1.0,1.0,430.0,3.0,420.0,2.0,2.0,2.0,...,"accepting cheap and premade,store bought foods",mechanic,turkish,"beef stroganoff, tacos, pizza","fresh fruits& vegetables, organic meats","healthy, fresh veggies,fruits & organic foods","grilled chicken ,stuffed shells,homemade chili",special education teacher,hockey,True
4,0.722222,190.00000,1.0,1.0,720.0,2.0,420.0,2.0,1.0,1.0,...,i have eaten generally the same foods but i do...,it,italian,"pasta, chicken tender, pizza","a lean protein such as grilled chicken, green ...",ideally i would like to be able to eat healthi...,"chicken parmesan, pulled pork, spaghetti and m...",substance abuse conselor,softball,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.722222,156.00000,1.0,1.0,610.0,4.0,420.0,2.0,2.0,3.0,...,i have noticed there is less time for a prepar...,accountant,italian,stromboli mac and cheese and pizza,mainly protein and vegetables with a complex c...,my ideal diet would consist of a majority of w...,"pasta, fish, steak",radiological technician,softball,True
121,0.444444,180.00000,1.0,1.0,265.0,2.0,315.0,2.0,2.0,3.0,...,eating pizza as an excuse when there is nothin...,doctor,mexican food,"isombe , plantains and ugali","a healthy meal is a variety of food , organic ...",eating home cooked meals everyday and being ab...,"fried rice ,baked potatoes ,curry chicken",public health advisor,basketball,True
122,0.934444,120.00000,1.0,1.0,720.0,3.0,420.0,1.0,2.0,3.0,...,less rice,ceo of company,korean,rice and potato,lots of vegetables,lots of veggies,"meat, rice, kimchi",real estate manageer,none,True
123,0.444444,135.00000,2.0,1.0,720.0,4.0,420.0,1.0,2.0,3.0,...,i don't eat as much on a daily basis since com...,store manager at giant eagle,italian,pizza and spaghetti,"a protein, a fruit, a starch, and a salad or s...",my ideal diet is the diet i am currently on. ...,"pizza, spaghetti, baked ziti",receptionist for a medical supply company,hockey,False


### Divisão do conjunto de dados

In [6]:
#Modelo de NB tratamento de Colunas não numericas (String)
def dropText(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df.drop(column, axis=1, inplace=True)
    return df

In [7]:
df2 = dropText(df2)

In [8]:
X_food = df2.drop(columns=['self_perception_overweight'], axis=1)
y_food = df2['self_perception_overweight']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_food, y_food, test_size=0.33, random_state=42)

### Seleção de algoritmos de classificação

 ## 1° Naive Bayes (NB) 
O algoritmo de Naïve Bayes no Scikit-learn é um método probabilístico, baseado no Teorema de Bayes, que assume independência entre as variáveis preditoras. Ele é eficiente para classificação de textos, filtragem de spam e outras tarefas. O Scikit-learn implementa variantes como GaussianNB, MultinomialNB e BernoulliNB.

 ## 2° k-Nearest Neighbors (kNN)
O algoritmo de Naïve Bayes no Scikit-learn é um método probabilístico, baseado no Teorema de Bayes, que assume independência entre as variáveis preditoras. Ele é eficiente para classificação de textos, filtragem de spam e outras tarefas. O Scikit-learn implementa variantes como GaussianNB, MultinomialNB e BernoulliNB.

### Treinamento e avaliação

In [28]:
#Naive Bayes (NB)
nb_model = GaussianNB()

nb_model.fit(X_train,y_train)

y_pred = nb_model.predict(X_test)


# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")


Acurácia: 0.67
Precisão: 0.67
Recall: 0.67
F1-score:0.67


In [29]:
#K-Nearest Neighbor (kNN)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report


model = KNeighborsClassifier(n_neighbors=3)

# Treinar o modelo
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.60
Precisão: 0.56
Recall: 0.60
F1-score:0.58


In [30]:
#Support Vector Machines (SVM)
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SVC(kernel='linear')

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.60
Precisão: 0.62
Recall: 0.60
F1-score:0.61


In [31]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='gini', random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.74
Precisão: 0.73
Recall: 0.74
F1-score:0.73


In [32]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
# Criar o modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.74
Precisão: 0.72
Recall: 0.74
F1-score:0.68


In [33]:
#Multilayer Perceptron
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(5, 3), activation='relu', max_iter=1000, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a precisão
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' para lidar com classes desbalanceadas

# Calcular o recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calcular o F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir os resultados
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.62
Precisão: 0.49
Recall: 0.62
F1-score:0.55


### Análise dos resultados